In [1]:
import pymongo
import pickle
from os import path

mongo_username= 'CEDC_Quigley'
mongo_password= 'lpP9pCcDxQ2Y0tv8'
mongo_server = 'mongodb+srv://ridgebaseaq-docdb-dqrxk.mongodb.net/test'
mongo_authSource = 'admin'
mongo_authMechanism = 'SCRAM-SHA-1'
mongo_database = 'legislative_documents'
mongo_people_collection = 'legiscan_people_data'
mongo_billmeta_collection = 'legiscan_bill_metadata'

client = pymongo.MongoClient(mongo_server
                        ,username=mongo_username
                        ,password=mongo_password
                        ,authSource=mongo_authSource
                        ,authMechanism=mongo_authMechanism)


data_file_location = 'C:/Users/William Mitchell/Documents/QuigleyData'

state_gov_nm = 'CA'
state_gov_id = 5

_pol_party = path.join(data_file_location,state_gov_nm + 'pol_party.dict')

_sponsor_table = path.join(data_file_location,state_gov_nm + 'sponsor_table.dict')

_temp_sponsor_table = path.join(data_file_location,state_gov_nm + 'temp_sponsor_table.dict')

_d_sponsor_table = path.join(data_file_location,state_gov_nm + 'd_sponsor_table.dict')
    
_d_temp_sponsor_table = path.join(data_file_location,state_gov_nm + 'd_temp_sponsor_table.dict')

#list for years to loop through looking for all possible sponsors
years = [2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020]



In [2]:
#create the political party dictionary
pol_party = {}

database = client[mongo_database]
collection = database[mongo_people_collection]
for l in collection.find({'state_id':state_gov_id}):
    pol_party[l['people_id']] = int(l['party_id'])

with open(_pol_party, 'wb') as f:
    pickle.dump(pol_party, f)

In [3]:
# data wrangling for time intervals

sponsor_table = {}
temp_sponsor_table = {}

database = client[mongo_database]
collection = database[mongo_billmeta_collection]

for t in years:
    temp_sponsor_table[t] ={'list_of_sponsors': []}
    sponsor_table[t] = {}

    for bill in collection.find({"$and": [{'state': state_gov_nm}, {'session.year_start': t}]}):
            sponsor_table[t][bill['bill_id']]={"primary": [], "co": [], "other": []}
            
            for s in bill['sponsors']:
                
                if s['sponsor_type_id'] == 1:
                    
                    sponsor_table[t][bill['bill_id']]['primary'].append(int(s['people_id']))
                
                    if not s['people_id'] in temp_sponsor_table[t]['list_of_sponsors']:
                        temp_sponsor_table[t]['list_of_sponsors'].append(int(s['people_id']))
                    
                    
                elif s['sponsor_type_id'] == 2:
                    sponsor_table[t][bill['bill_id']]['co'].append(int(s['people_id']))
                    
                    if not s['people_id'] in temp_sponsor_table[t]['list_of_sponsors']:
                        temp_sponsor_table[t]['list_of_sponsors'].append(int(s['people_id']))
                else:
                    sponsor_table[t][bill['bill_id']]['other'].append(int(s['people_id']))

with open(_sponsor_table, 'wb') as f:
    pickle.dump(sponsor_table, f)

with open(_temp_sponsor_table, 'wb') as f:
    pickle.dump(temp_sponsor_table, f)

In [4]:
# data wrangling for district and time

d_sponsor_table = {}
d_temp_sponsor_table = {}


database = client[mongo_database]
collection = database[mongo_billmeta_collection]

for t in years:
    
    d_temp_sponsor_table[t] ={'list_of_sponsors': []}
    d_sponsor_table[t] = {}

    for bill in collection.find({"$and": [{'state': state_gov_nm}, {'session.year_start': t}]}):
            d_sponsor_table[t][bill['bill_id']]={"primary": [], "co": [], "other": []}
            
            for s in bill['sponsors']:
                
                if s['sponsor_type_id'] == 1:
                    
                    d_sponsor_table[t][bill['bill_id']]['primary'].append(s['district'])
                
                    if not s['district'] in d_temp_sponsor_table[t]['list_of_sponsors']:
                        d_temp_sponsor_table[t]['list_of_sponsors'].append(s['district'])
                    
                    
                elif s['sponsor_type_id'] == 2:
                    d_sponsor_table[t][bill['bill_id']]['co'].append(s['district'])
                    
                    if not s['district'] in d_temp_sponsor_table[t]['list_of_sponsors']:
                        d_temp_sponsor_table[t]['list_of_sponsors'].append(s['district'])
                else:
                    d_sponsor_table[t][bill['bill_id']]['other'].append(s['district'])

with open(_d_sponsor_table, 'wb') as f:
    pickle.dump(d_sponsor_table, f)
    
with open(_d_temp_sponsor_table, 'wb') as f:
    pickle.dump(d_temp_sponsor_table, f)